In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
early_stopping_monitor = EarlyStopping(patience=2)

In [ ]:
#test=pd.io.parsers.read_csv("test.csv")

In [4]:
train=pd.io.parsers.read_csv("train.csv", nrows = 20000000)

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train.describe()

,ID,Carrier,ConversionPayOut,advertiserCampaignId,Fraud
count,2.000000e+07,2.000000e+07,2.000000e+07,1.999964e+07,2.000000e+07
mean,1.000000e+07,9.218364e+01,2.455294e-02,7.767680e+03,4.850000e-06
std,5.773503e+06,2.031770e+02,5.596538e+00,4.224424e+03,2.202266e-03
min,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+06,-1.000000e+00,0.000000e+00,4.901000e+03,0.000000e+00
50%,1.000000e+07,5.000000e+00,0.000000e+00,7.510000e+03,0.000000e+00
75%,1.500000e+07,7.400000e+01,0.000000e+00,1.133800e+04,0.000000e+00
max,2.000000e+07,1.267000e+03,8.000000e+03,1.711100e+04,1.000000e+00


In [6]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [7]:
train1 = train.drop(['ConversionDate','ConversionStatus','ID', 'Browser', 'Device', 'ClickDate','UserIp','Fraud', 'OS', 'TrafficType'], axis=1)

In [8]:
#train1 = pd.get_dummies(data=train1, columns=['OS','TrafficType'])
train1['RefererUrl'] = number.fit_transform(train1['RefererUrl'].astype('str'))
train1['Country'] = number.fit_transform(train1['Country'].astype('str'))
train1['subPublisherId'] = number.fit_transform(train1['subPublisherId'].astype('str'))
train1['publisherId'] = number.fit_transform(train1['publisherId'].astype('str'))

In [9]:
train1[['Country','RefererUrl','publisherId','subPublisherId','advertiserCampaignId','Carrier']] = train1[['Country','RefererUrl','publisherId','subPublisherId','advertiserCampaignId','Carrier']].replace(0, np.NaN)
# drop rows with missing values
train1.dropna(inplace=True)
# summarize the number of rows and columns in the dataset
print(train1.shape)

(19935442, 7)


In [10]:
y=train1['ConversionPayOut'].ravel()

In [11]:
train1=train1.drop('ConversionPayOut', axis=1)

In [12]:
x = train1.values

In [ ]:
#t1 = test.drop(['ID', 'Browser', 'Device', 'ClickDate','UserIp','Fraud'], axis=1)
##t1 = pd.get_dummies(data=t1, columns=['OS','TrafficType'])
#t1['RefererUrl'] = number.fit_transform(t1['RefererUrl'].astype('str'))
#t1['Country'] = number.fit_transform(t1['Country'].astype('str'))
#t1['subPublisherId'] = number.fit_transform(t1['subPublisherId'].astype('str'))
#t1['publisherId'] = number.fit_transform(t1['publisherId'].astype('str'))
#testA = t1.values

In [41]:
model = Sequential()
model.add(Dense(6, input_shape=(6,), activation= 'relu'))
model.add(Dense(2, activation= 'relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [42]:
model.fit(x,y,validation_split = 0.1, epochs = 2, callbacks = [early_stopping_monitor])

Train on 17941897 samples, validate on 1993545 samples
Epoch 1/2
17941897/17941897 [==============================] - 711s - loss: 1364.1839 - val_loss: 30.2869
Epoch 2/2
17941897/17941897 [==============================] - 680s - loss: 31.0722 - val_loss: 30.2870


In [43]:
from keras.models import load_model
model.save('model_file.h5')

In [44]:
testfinal=pd.io.parsers.read_csv("testfinal.csv")
testA = testfinal.values

In [45]:
predictsA = model.predict(testA)

In [46]:
i1 = pd.DataFrame(predictsA)

In [47]:
i1.columns = ['ConversionPayOut']

In [48]:
i1.describe()

,ConversionPayOut
count,2.554848e+07
mean,5.130325e-03
std,1.582732e-03
min,7.019817e-03
25%,7.019817e-03
50%,7.019817e-03
75%,7.019817e-03
max,7.019817e-03


In [59]:
i1 = i1.fillna(0)

In [60]:
i1.isnull().sum()

ConversionPayOut    0
dtype: int64

In [61]:
ty = pd.io.parsers.read_csv("ID.csv")
i2 = ty
result = pd.concat([i2, i1], axis=1)

In [ ]:
result.tail()

,ID,ConversionPayOut
25548868,88916157,0.00702
25548869,88916158,0.00702
25548870,88916159,0.00702
25548871,88916160,0.00702
25548872,88916161,0.00702


In [ ]:
result.to_csv("result.csv", index=False)